In [21]:
import numpy as np
import data_preprocessing
import PyEDCR
import vit_pipeline

# Creating test_data

In [6]:
# test_pred_fine = np.load('test/test_pred_fine.npy')
# test_pred_coarse = np.load('test/test_pred_coarse.npy')

train_fine_pred = np.load('combined_results/vit_b_16_train_fine_pred_BCE_lr0.0001_e19.npy')
train_coarse_pred = np.load('combined_results/vit_b_16_train_coarse_pred_BCE_lr0.0001_e19.npy')
test_fine_pred = np.load('combined_results/vit_b_16_test_fine_pred_BCE_lr0.0001_e19.npy')
test_coarse_pred = np.load('combined_results/vit_b_16_test_coarse_pred_BCE_lr0.0001_e19.npy')

test_fine_true, test_coarse_true = data_preprocessing.get_ground_truths(test=True,)
train_fine_true, train_coarse_true = data_preprocessing.get_ground_truths(test=False)

In [7]:
import numpy as np

# Initialize empty lists to store indices for each value
indices_list_train = []

# Iterate through each unique value and limit to 10 instances
for value in [0, 1, 2]:
    # Get indices where the value is present
    indices = np.where(train_fine_true == value)[0]
    
    # Limit to 10 instances and shuffle the indices
    indices = np.random.choice(indices, size=10, replace=False)
    
    # Append the limited indices to corresponding lists
    indices_list_train.extend(indices)

In [8]:
import numpy as np

# Initialize empty lists to store indices for each value
indices_list_test = []

# Iterate through each unique value and limit to 10 instances
for value in [0, 1, 2]:
    # Get indices where the value is present
    indices = np.where(test_fine_true == value)[0]
    
    # Limit to 10 instances and shuffle the indices
    indices = np.random.choice(indices, size=10, replace=False)
    
    # Append the limited indices to corresponding lists
    indices_list_test.extend(indices)

In [9]:
check_train_fine_pred = train_fine_pred[indices_list_train]
check_train_coarse_pred = train_coarse_pred[indices_list_train]

check_test_fine_pred = test_fine_pred[indices_list_test]
check_test_coarse_pred = test_coarse_pred[indices_list_test]

check_train_fine_true = train_fine_true[indices_list_train]
check_train_coarse_true = train_coarse_true[indices_list_train]

check_test_fine_true = test_fine_true[indices_list_test]
check_test_coarse_true = test_coarse_true[indices_list_test]

In [13]:
np.save("test_data/check_train_fine_pred.npy", check_train_fine_pred)
np.save("test_data/check_train_coarse_pred.npy", check_train_coarse_pred)

np.save("test_data/check_test_fine_pred.npy", check_test_fine_pred)
np.save("test_data/check_test_coarse_pred.npy", check_test_coarse_pred)

np.save("test_data/check_train_fine_true.npy", check_train_fine_true)
np.save("test_data/check_train_coarse_true.npy", check_train_coarse_true)

np.save("test_data/check_test_fine_true.npy", check_test_fine_true)
np.save("test_data/check_test_coarse_true.npy", check_test_coarse_true)

# Retrieve check data

In [9]:
test_true_fine_data = np.load(r'data/test_fine/test_true_fine.npy')
test_true_coarse_data = np.load(r'data/test_coarse/test_true_coarse.npy')

train_true_fine_data = np.load(r'data/train_fine/train_true_fine.npy')
train_true_coarse_data = np.load(r'data/train_coarse/train_true_coarse.npy')

In [14]:
np.where(train_true_fine_data==1)

In [19]:
labels = data_preprocessing.fine_grain_classes_str
# Sample data (sorted)
data = test_true_fine_data

# Define the target label
for index, target_label in enumerate(labels):

    # Find the starting and ending indices of the target label
    start_index = np.argmax(test_true_fine_data==index)
    end_index = start_index + 1

    # While the next label is the same, keep incrementing the end index
    while end_index < len(data) and data[end_index] == index:
        end_index += 1

    # Print the range of indices
    print(f"{target_label}: [{start_index}, {end_index - 1}]")

In [7]:
main_model_name = 'vit_b_16'
combined = True
test = True
g_str = data_preprocessing.granularities['fine'].g_str
loss = 'BCE'
lr = 1e-04
pred=False
num_epochs=19
test_true_fine_path = vit_pipeline.get_filepath(model_name=main_model_name,
                          combined=combined,
                          test=test,
                          granularity=g_str,
                          loss=loss,
                          lr=lr,
                          pred=True,
                          epoch=num_epochs)

In [8]:
test_true_fine_data = np.load(test_true_fine_path)

In [24]:
data_preprocessing.coarse_to_fine

In [34]:
g_fine, g_coarse = data_preprocessing.granularities.values()
fg_l, cg_l = list(data_preprocessing.fine_grain_labels.values()), list(data_preprocessing.coarse_grain_labels.values())


In [32]:
import numpy as np

# Sample data (assuming fg_l and cg_l are defined elsewhere)
# check_train_fine_pred = np.array([0, 2, 1])
# check_train_coarse_pred = np.array([0, 1, 0])
# check_train_fine_true = np.array([2, 1, 0])
# check_train_coarse_true = np.array([1, 0, 2])
# fg_l = ["2S19_MSTA", "BMP-3", "T-72B"]
# cg_l = ["Tank", "Infantry fighting vehicle", "Self Propelled Artillery"]

# Function to format prediction/ground truth pairs
def format_pair(pred_fine, pred_coarse, true_fine, true_coarse):
  max_pred_len = max([len(str(l)) for l in fg_l]) + max([len(str(l)) for l in cg_l]) + 10 
  max_true_len = max([len(str(l)) for l in fg_l]) + max([len(str(l)) for l in cg_l]) + 10
  pred_str = f"pred: ({fg_l[pred_fine]}, {cg_l[pred_coarse]})"
  true_str = f"true: ({fg_l[true_fine]}, {cg_l[true_coarse]})"
  pred_str += " " * (max_pred_len - len(pred_str))
  true_str += " " * (max_true_len - len(true_str))
  return f"pred: ({fg_l[pred_fine]}, {cg_l[pred_coarse]})", f"true: ({fg_l[true_fine]}, {cg_l[true_coarse]})"

# Print information in desired format
for i in range(len(check_train_fine_pred)):
  pred_fine, pred_coarse = check_train_fine_pred[i], check_train_coarse_pred[i]
  true_fine, true_coarse = check_train_fine_true[i], check_train_coarse_true[i]
  pred_str, true_str = format_pair(pred_fine, pred_coarse, true_fine, true_coarse)
  print(f'["{pred_str}", "{true_str}"]')



In [ ]:
!python test/test_get_where_train_fp_l.py